## Function to read json rows into a dataframe
json_df = pd.DataFrame.from_records(df['edp_raw_data_map'].apply(json.loads))

In [1]:
import  plotly.express as px
import pandas as pd
import pyodbc
import json
from datetime import datetime 

def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')
pd.options.display.max_columns = 100
pd.options.display.max_rows = 58

# Here is a function I use to read a file with a sql query
def file_to_string(fileName):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string

# Then I use this to run the query and get the results as a dataframe
def run_query(query, dsn='edp-workbench-cshub'):
    print("Start Time "+timestamp())
    cnxn = pyodbc.connect('DSN='+dsn, autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    print("End Time "+timestamp())
    return df


In [20]:
df_t=run_query(file_to_string('SQL/DailyMetricsCRT-AHT.SQL'))
df_t

Start Time 2023-12-13 03:29:46 PM


C:\Users\austen.moore\AppData\Local\Temp\ipykernel_5580\131032485.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


End Time 2023-12-13 03:30:18 PM


,Employee,Date,work_type,HoursWorked,Sessions,Total_Resolution_Time,hero_solve_score,hero_solve_count,Helix Searches,Helix Sessions,Total_Eligible,Total_Accepted
0,572909,2023-07-24,PPX,7.00,17,16860,24.0,14.93,2.0,2.0,13.0,0.0
1,548646,2023-09-11,PPX,3.42,12,11793,13.0,11.25,2.0,1.0,11.0,0.0
2,548646,2023-06-13,PPX,8.06,27,24083,29.0,21.99,1.0,1.0,14.0,3.0
3,547381,2023-10-09,Hourly,3.43,5,11331,7.0,5.33,3.0,1.0,5.0,0.0
4,574166,2023-06-06,PPX,5.70,16,16199,20.0,17.34,5.0,3.0,13.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3705,426097,2023-11-06,PPX,7.63,16,23384,17.0,13.50,5.0,2.0,12.0,1.0
3706,393198,2023-06-14,PPX,4.13,5,10765,9.0,6.08,3.0,1.0,6.0,1.0
3707,573276,2023-10-27,PPX,6.79,12,20742,19.0,17.58,1.0,1.0,5.0,2.0
3708,547655,2023-10-25,PPX,5.68,13,15899,14.0,12.00,NaN,NaN,9.0,0.0


In [7]:
PreJoinTest = run_query(
"""
SELECT
    
FROM 
    hive.care.l3_asurion_twilio_interaction_interval_summary
GROUP BY measure_name
""")
PreJoinTest.head()

Start Time 2023-12-13 03:18:15 PM


C:\Users\austen.moore\AppData\Local\Temp\ipykernel_5580\131032485.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


End Time 2023-12-13 03:18:18 PM


,measure_name
0,session_duration_sec
1,active_experts_ct
2,abandoned time
3,expert_sla_denom
4,wrap time


In [12]:
run_query("""SELECT """)

,dw_soluto_crm_id,dw_date_id,dw_client_id,dw_client_channel_id,dw_agent_master_id,dw_customer_case_id,dw_subscriber_id,client_name,support_session_id,contact_id,parent_support_session_id,linked_support_session_id,linked_contact_id,contact_group,interaction_id,start_dt,end_dt,duration_sec,platform_name,channel_name,user_name,account_number_name,mdn_name,ani_name,subscriber_uuid,program_name,enrollment_status,end_call_reason,end_call_sub_reason,call_back_reason,call_back_type,call_reason,sub_reason,issue_type,issue_status,notes_desc,make_name,model_name,os_name,nick_name,call_direction_desc,contact_source_name,ivr_validation_flg,text_app_flg,connected_flg,non_connected_flg,cvp_call_id,icm_router_day,icm_router_key,tte_flg,smb_flg,account_category,account_type_name,customer_type_name,source,dw_create_dt,dw_update_dt,fcr_eligible,fcr_pass,upsell_accepted,upsell_desc,start_dt_utc,end_dt_utc,clndr_dt_cst,edp_created_dttm,edp_updated_dttm,batchid,current_persona,cancellation_request_reason_selected,cancellation_request_subreason_selected,cancellation_request_rebuttals_selected_list,rebuttalcode1,rebuttalcode2,enrollment_channel,planid,Employee


In [13]:
run_query("""
    SELECT
        CAST(eh.expert_id AS BIGINT) as "Employee",
        eh.tenure_group as "TenureGroup",
        eh.*,
        em.expert_full_name as "Employee Name"
    FROM
        hive.care.l3_asurion_whole_home_expert_hierarchy eh
        LEFT OUTER JOIN
        hive.care.l3_asurion_whole_home_expert_master em
            ON eh.expert_id = em.expert_id
    WHERE 1=1
        AND UPPER(eh.business_unit) = 'PSS'
        AND eh.location = 'flor'""")

C:\Users\austen.moore\AppData\Local\Temp\ipykernel_25172\1858826741.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


,Employee,TenureGroup,expert_id,agent_id,eff_start_dt,eff_end_dt,supervisor_id,supervisor_first_name,supervisor_last_name,supervisor_full_name,manager_id,manager_first_name,manager_last_name,manager_full_name,senior_manager_id,senior_manager_first_name,senior_manager_last_name,senior_manager_full_name,director_id,director_first_name,director_last_name,director_full_name,business_unit,lob_name,site_name,tenure_group,location,location_descr,external_loc_id,external_loc_name,region,market,jobcode,current_flg,source,edp_created_dttm,edp_updated_dttm,batchid,std_year,std_month,Employee Name
0,381967,180+,381967,-986180515490,2021-10-01,2021-10-11,395446,kevin,bowers,kevin bowers,396306,nicole,paterka,nicole paterka,None,None,None,None,None,None,None,None,pss,pss-at&t,pitbulls,180+,flor,orlando,None,None,None,None,crn1125,n,aspect,2021-12-06 00:00:00.000,2021-12-06 00:00:00.000,1638808873,2021,10,moses bien-aime
1,381967,180+,381967,-986180515490,2021-10-12,2021-10-19,395446,kevin,bowers,kevin bowers,396306,nicole,paterka,nicole paterka,None,None,None,None,None,None,None,None,pss,pss-at&t,pitbulls,180+,flor,orlando,None,None,None,None,crn1125,n,aspect,2021-12-06 00:00:00.000,2021-12-06 00:00:00.000,1638808873,2021,10,moses bien-aime
2,270166,180+,270166,-988667352541,2021-10-03,2021-10-04,113148,kristen,kelly,kristen kelly,303544,anthony,rodriguez,anthony rodriguez,None,None,None,None,None,None,None,None,pss,pss-undefined,glowzilla,180+,flor,orlando,None,None,None,None,pss2010,n,aspect,2021-12-06 00:00:00.000,2021-12-06 00:00:00.000,1638808873,2021,10,jonathan freeman
3,270166,180+,270166,-988667352541,2021-10-01,2021-10-02,113148,kristen,kelly,kristen kelly,303544,anthony,rodriguez,anthony rodriguez,None,None,None,None,None,None,None,None,pss,pss-undefined,glowzilla,180+,flor,orlando,None,None,None,None,pss2010,n,aspect,2021-12-06 00:00:00.000,2021-12-06 00:00:00.000,1638808873,2021,10,jonathan freeman
4,396422,180+,396422,-979981900449,2021-10-12,2021-10-12,276497,wendy,rodriguez,wendy rodriguez,132171,shaine,hobdy,shaine hobdy,None,None,None,None,None,None,None,None,pss,pss-verizon wireless,glowzilla,180+,flor,orlando,None,None,None,None,pss1002,n,aspect,2021-12-06 00:00:00.000,2021-12-06 00:00:00.000,1638808873,2021,10,marilyn padilla-portela
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175374,568444,0-30,568444,-977831197889,2023-02-05,2023-02-22,425503,carmen,baker,carmen baker,396306,nicole,paterka,nicole paterka,303544,anthony,rodriguez,anthony rodriguez,None,None,None,None,pss,pss-verizon wireless,mustangs trn,0-30,flor,orlando,None,None,None,None,crn1119,n,aspect,2023-03-30 10:31:31.714,2023-03-30 10:31:31.714,1680172291,2023,2,michael murray
175375,552704,180+,552704,-978113116718,2023-02-01,2023-02-28,269489,gilissa,rosario,gilissa rosario,111144,lala,crawford,lala crawford,303544,anthony,rodriguez,anthony rodriguez,None,None,None,None,pss,pss-verizon wireless,mustangs,180+,flor,orlando,None,None,None,None,crn1119,n,aspect,2023-03-30 10:31:31.714,2023-03-30 10:31:31.714,1680172291,2023,2,michael harrison
175376,568671,0-30,568671,-977831198955,2023-02-01,2023-02-03,299210,giovanni,lockwood,giovanni lockwood,396306,nicole,paterka,nicole paterka,303544,anthony,rodriguez,anthony rodriguez,None,None,None,None,pss,pss-verizon wireless,mustangs trn,0-30,flor,orlando,None,None,None,None,crn1119,n,aspect,2023-03-30 10:31:31.714,2023-03-30 10:31:31.714,1680172291,2023,2,james cook
175377,568549,0-30,568549,-977831196656,2023-02-01,2023-02-04,299210,giovanni,lockwood,giovanni lockwood,396306,nicole,paterka,nicole paterka,303544,anthony,rodriguez,anthony rodriguez,None,None,None,None,pss,pss-verizon wireless,mustangs trn,0-30,flor,orlando,None,None,None,None,crn1119,n,aspect,2023-03-30 10:31:31.714,2023-03-30 10:31:31.714,1680172291,2023,2,margaux escalon smith
